Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [82]:
number_of_batches_when_reduced = 3
batch_size = 128

def run_model(num_steps, reduced=False):
    actual_train_dataset = train_dataset
    actual_train_labels = train_labels
    if (reduced):
        actual_train_dataset = train_dataset[:number_of_batches_when_reduced * batch_size, :]
        actual_train_labels = train_labels[:number_of_batches_when_reduced * batch_size, :]
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (actual_train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = actual_train_dataset[offset:(offset + batch_size), :]
        batch_labels = actual_train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f, training acc %.1f%%, validation acc %.1f%%" % (step, l, accuracy(predictions, batch_labels), accuracy(
            valid_prediction.eval(), valid_labels) ))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [43]:
# Logistic SGD

beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    (tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +
        beta * (tf.nn.l2_loss(weights)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
run_model(10001)

Initialized
Minibatch loss at step 0: 18.536770, 14.8%, 14.9%
Minibatch loss at step 500: 2.742246, 73.4%, 76.0%
Minibatch loss at step 1000: 1.650719, 78.1%, 78.0%
Minibatch loss at step 1500: 1.400585, 78.9%, 79.7%
Minibatch loss at step 2000: 0.898838, 82.8%, 81.4%
Minibatch loss at step 2500: 0.834700, 85.2%, 81.2%
Minibatch loss at step 3000: 0.910054, 78.9%, 82.4%
Minibatch loss at step 3500: 0.843195, 81.2%, 81.1%
Minibatch loss at step 4000: 0.736633, 82.0%, 81.8%
Minibatch loss at step 4500: 0.618556, 84.4%, 81.7%
Minibatch loss at step 5000: 0.553512, 85.9%, 82.6%
Minibatch loss at step 5500: 0.757775, 81.2%, 82.2%
Minibatch loss at step 6000: 0.796296, 82.0%, 82.2%
Minibatch loss at step 6500: 0.701299, 82.8%, 82.6%
Minibatch loss at step 7000: 0.832602, 78.1%, 82.5%
Minibatch loss at step 7500: 0.550338, 85.9%, 82.4%
Minibatch loss at step 8000: 0.647765, 82.0%, 82.8%
Minibatch loss at step 8500: 0.867115, 77.3%, 81.4%
Minibatch loss at step 9000: 0.799332, 81.2%, 82.7%
Min

In [47]:
hidden_nodes = 1024
beta = 0.001
def neural_network():
    def model(x, weights1, biases1, weights2, biases2):
        layer_1 = tf.matmul(x, weights1) + biases1
        layer_1_relu = tf.nn.relu(layer_1)
        return tf.matmul(layer_1_relu, weights2) + biases2

    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
      biases1 = tf.Variable(tf.zeros([hidden_nodes]))

      weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
      biases2 = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = model(tf_train_dataset, weights1, biases1, weights2, biases2)
      loss = tf.reduce_mean(
        (tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))  +
            beta * (tf.nn.l2_loss(weights1) +
                    tf.nn.l2_loss(weights2)))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights1, biases1, weights2, biases2))
      test_prediction =  tf.nn.softmax(model(tf_test_dataset, weights1, biases1, weights2, biases2))

In [66]:
neural_network()
run_model(3001)

Initialized
Minibatch loss at step 0: 663.357910, training acc 5.5%, validation acc 32.6%
Minibatch loss at step 500: 199.302185, training acc 78.9%, validation acc 78.6%
Minibatch loss at step 1000: 115.231857, training acc 82.8%, validation acc 82.6%
Minibatch loss at step 1500: 69.016640, training acc 82.0%, validation acc 82.9%
Minibatch loss at step 2000: 41.403770, training acc 85.2%, validation acc 85.3%
Minibatch loss at step 2500: 25.164795, training acc 90.6%, validation acc 86.4%
Minibatch loss at step 3000: 15.459551, training acc 89.1%, validation acc 86.8%
Test accuracy: 92.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [83]:
neural_network()
run_model(3001, True)

Initialized
Minibatch loss at step 0: 941.361633, training acc 9.4%, validation acc 26.7%
Minibatch loss at step 500: 191.497086, training acc 100.0%, validation acc 76.2%
Minibatch loss at step 1000: 116.156067, training acc 100.0%, validation acc 76.0%
Minibatch loss at step 1500: 70.456291, training acc 100.0%, validation acc 76.4%
Minibatch loss at step 2000: 42.731045, training acc 100.0%, validation acc 76.4%
Minibatch loss at step 2500: 25.915590, training acc 100.0%, validation acc 76.9%
Minibatch loss at step 3000: 15.717144, training acc 100.0%, validation acc 76.8%
Test accuracy: 82.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
hidden_nodes = 1024
beta = 0.001
keep_prob = 0.5
def model(x, weights1, biases1, weights2, biases2, for_training=False):
    layer_1 = tf.matmul(x, weights1) + biases1
    layer_1_relu = tf.nn.relu(layer_1)
    if (for_training):
        layer_1_relu = tf.nn.dropout(layer_1_relu, keep_prob)
    return tf.matmul(layer_1_relu, weights2) + biases2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))

  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = model(tf_train_dataset, weights1, biases1, weights2, biases2, True)
  loss = tf.reduce_mean(
    (tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))  +
        beta * (tf.nn.l2_loss(weights1) +
                tf.nn.l2_loss(weights2)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction =  tf.nn.softmax(model(tf_test_dataset, weights1, biases1, weights2, biases2))

run_model(3001, True)

Initialized
Minibatch loss at step 0: 831.162109, training acc 6.2%, validation acc 31.2%
Minibatch loss at step 500: 191.890976, training acc 100.0%, validation acc 75.6%
Minibatch loss at step 1000: 116.409416, training acc 100.0%, validation acc 76.4%
Minibatch loss at step 1500: 70.606720, training acc 100.0%, validation acc 76.6%
Minibatch loss at step 2000: 42.824017, training acc 100.0%, validation acc 76.9%
Minibatch loss at step 2500: 25.971579, training acc 100.0%, validation acc 77.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
